이 파일을 6가지 셀로 구성되어 있음.  
1. 각 셀의 역할을 설명하는 이 텍스트셀.
2. 임시 파일을 코랩에 업로드하는 코드셀 (임시 파일: tagging_results2.csv)
3. 임시 파일을 불러오는 코드셀 (임시 파일이란 데이터베이스를 대체하는 파일 의미)
4. 필요한 모듈(ipycytoscape pandas)을 설치하는 코드셀
5. 코랩에서 서드파티 위젯 활성화하는 코드셀
6. 네트워크 그래프를 그리는 코드셀.
- 이 중 2, 5번 셀은 코랩에서 실행 시에만 필요. 즉, 주피터 환경의 경우 실행할 필요 X   
- 필요한 모듈을 설치하신 뒤 가장 마지막에 있는 네트워크를 그리는 코드 셀을 백엔드와 연결하시면 됩니다.

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving tagging_results2.csv to tagging_results2 (1).csv


In [ ]:
# 파일 확인
import pandas as pd

data = pd.read_csv("tagging_results2.csv", encoding = "cp949")
data.head()

,id,session_id,username,report_tags,counsel_tags,tag_reasons,timestamp
0,1,89b75489-abf6-47ad-9f8b-d5721f1a99d9,test123,"[""신체학대""]","[""아동 신체학대""]","태깅 결과: {""tags"": [""신체학대""]}\n\n- 신체학대: 아빠가 화가 나서...",48:12.1
1,2,89b75489-abf6-47ad-9f8b-d5721f1a99d9,test123,"[""신체학대""]","[""아동 신체학대""]","태깅 결과: {""tags"": [""신체학대""]}\n\n- 신체학대: 아빠가 화가 나서...",02:50.1
2,3,355982c2-4256-48f2-890d-84581d8380b0,test123,"[""신체학대""]","[""아동 신체학대""]","태깅 결과: {""tags"": [""신체학대""]}\n\n- 신체학대: 아빠가 화가 나서...",52:11.8
3,4,355982c2-4256-48f2-890d-84581d8380b0,test123,"[""신체학대"", ""정서학대""]","[""아동 신체학대"", ""아동 정서학대""]","태깅 결과: {""tags"": [""신체학대"", ""정서학대""]}\n\n- 신체학대: 아...",54:23.5
4,5,355982c2-4256-48f2-890d-84581d8380b0,test123,"[""신체학대""]","[""아동 신체학대""]","태깅 결과: {""tags"": [""신체학대""]}\n\n- 신체학대: 아빠가 화가 나서...",05:11.2


In [ ]:
# 모듈 설치
!pip install ipycytoscape pandas

In [ ]:
# 1) 설치 (세션마다 한 번)
!pip -q install ipywidgets==7.7.1 ipycytoscape

# 2) 코랩에서 서드파티 위젯 활성화
from google.colab import output
output.enable_custom_widget_manager()


In [ ]:
import pandas as pd
import ast
from collections import Counter
from ipycytoscape import CytoscapeWidget

# ===== 설정 =====
CSV_PATH = "tagging_results2.csv"  # 너가 올려둔 파일명
CHILD_COL_CANDIDATES = ["username", "user", "child_id", "id", "아이디"]
TAG_COL_PREF_ORDER   = ["report_tags", "보고서용태그", "tags", "counsel_tags", "태그"]

# 🔧 노드 크기 정책
CHILD_SIZE = 22           # 아동 노드: 고정 크기
TAG_SIZE_MIN = 30         # 태그 노드: 최소 크기(고정)
TAG_SIZE_MAX = 60         # 태그 노드: 최대 크기(고정)

def parse_tags(v):
    """문자열 '["태그1","태그2"]' 또는 '태그1, 태그2' 모두 처리"""
    if pd.isna(v):
        return []
    s = str(v).strip()
    try:
        out = ast.literal_eval(s)
        if isinstance(out, (list, tuple)):
            return [str(x).strip() for x in out if str(x).strip()]
    except Exception:
        pass
    if "," in s:
        return [t.strip() for t in s.split(",") if t.strip()]
    return [s] if s else []

# ===== CSV 읽기 =====
df = pd.read_csv(CSV_PATH, encoding="cp949", engine="python")

# 컬럼 자동 탐지
child_col = next((c for c in CHILD_COL_CANDIDATES if c in df.columns), None)
tag_col   = next((c for c in TAG_COL_PREF_ORDER   if c in df.columns), None)

if child_col is None or tag_col is None:
    raise ValueError(
        f"필요 컬럼을 찾지 못했어요. 아동 후보={CHILD_COL_CANDIDATES}, 태그 후보={TAG_COL_PREF_ORDER}\n"
        f"현재 컬럼: {list(df.columns)}"
    )

# 태그 리스트화
df["_tags_list"] = df[tag_col].apply(parse_tags)

# ===== 노드/엣지 빌드 =====
# 아이디/태그가 동일 문자일 때 충돌 방지용 prefix
def cid(x):  # child id
    return f"child::{x}"
def tid(x):  # tag id
    return f"tag::{x}"

child_ids = sorted(set(df[child_col].astype(str)))
tag_set   = set()
edges_set = set()

for _, row in df.iterrows():
    c = str(row[child_col])
    for t in row["_tags_list"]:
        t = str(t)
        if not t:
            continue
        tag_set.add(t)
        edges_set.add((cid(c), tid(t)))  # 유일 엣지

# ✅ 태그 노드 degree 계산 (연결된 엣지 수)
tag_degree = Counter()
for c, t in edges_set:
    # t는 'tag::...' 형태
    tag_degree[t] += 1

# 태그 degree 범위 (mapData용)
if tag_degree:
    deg_min_tag = min(tag_degree.values())
    deg_max_tag = max(tag_degree.values())
else:
    deg_min_tag, deg_max_tag = 1, 1  # 비어있을 때 안전값

# ===== ipycytoscape 요소 구성 =====
nodes = []

# 아동 노드: 라벨 숨김 + 크기 고정 (degree는 쓰지 않지만 1로 넣어둠)
for c in child_ids:
    nodes.append({
        "data": {"id": cid(c), "type": "child", "label": "", "degree": 1}
    })

# 태그 노드: 라벨 표시 + degree 포함 (크기 맵핑에 사용)
for t in sorted(tag_set):
    t_id = tid(t)
    nodes.append({
        "data": {
            "id": t_id,
            "type": "tag",
            "label": t,
            "degree": int(tag_degree.get(t_id, 1))
        }
    })

edges = [{"data": {"source": c, "target": t}} for (c, t) in sorted(edges_set)]

# ===== 그래프 위젯 =====
cy = CytoscapeWidget()

# ✅ 디자인 강화 (레이아웃 영향 없음)
# - 아동 노드: 고정 크기 CHILD_SIZE
# - 태그 노드: degree 기반 mapData로 TAG_SIZE_MIN ~ TAG_SIZE_MAX 사이에서 가변
#   (deg_min_tag == deg_max_tag일 때도 안전하게 동작)
cy.set_style([
    # 기본 투명도 살짝 낮춰 포커스 대비 확보
    {"selector": "node, edge", "style": {"opacity": 0.8}},

    # 노드 공통
    {
        "selector": "node",
        "style": {
            "label": "data(label)",
            "font-size": "12px",
            "min-zoomed-font-size": 8,
            "text-outline-color": "#ffffff",
            "text-outline-width": 2,
            "color": "#111",
            "border-width": 1,
            "border-color": "#999",
            "shape": "ellipse"
        }
    },

    # 아동 노드: 파랑톤 + 라벨 숨김 + 크기 고정
    {
        "selector": 'node[type = "child"]',
        "style": {
            "background-color": "#4C7DFF",
            "border-color": "#2b5bd9",
            "label": "",
            "width":  CHILD_SIZE,
            "height": CHILD_SIZE
        }
    },

    # 태그 노드: 노랑톤 + degree 기반 크기 맵핑
    # (deg_min_tag == deg_max_tag면 내부적으로 동일 크기 적용)
    {
        "selector": 'node[type = "tag"]',
        "style": {
            "background-color": "#F6BD16",
            "border-color": "#d09c00",
            "width":  f"mapData(degree, {deg_min_tag}, {max(deg_max_tag, deg_min_tag)}, {TAG_SIZE_MIN}, {TAG_SIZE_MAX})",
            "height": f"mapData(degree, {deg_min_tag}, {max(deg_max_tag, deg_min_tag)}, {TAG_SIZE_MIN}, {TAG_SIZE_MAX})",
        }
    },

    # 엣지: 얇고 부드럽게
    {
        "selector": "edge",
        "style": {
            "line-color": "#bfc4cc",
            "width": 1.1,
            "curve-style": "bezier",
            "opacity": 0.65
        }
    },

    # 호버/선택 시 이웃 강조
    {"selector": "node:hover", "style": {"opacity": 1.0, "border-width": 2, "border-color": "#222"}},
    {"selector": "node:selected", "style": {"opacity": 1.0, "border-width": 2, "border-color": "#222"}},
    {"selector": "node:selected ~ edge", "style": {"opacity": 0.95, "line-color": "#666"}},
])

# 요소 주입
cy.graph.add_graph_from_json({"nodes": nodes, "edges": edges})

# ✅ 레이아웃은 '원본 그대로' 유지
cy.set_layout(name="cose")

# 인터랙션
cy.user_panning_enabled = True
cy.user_zooming_enabled = True

cy  # 노트북에서 위젯으로 바로 표시됨


CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node, edge', 'style': {'opac…